## Abstract:

The aim of this notebook is to show to create an ISA document for depositing Stable Isotope Resolved Metabolomics Study metadata using the ISA API.

This notebook highlights key steps of the deposition, including:
- declaration of study variables and treatment groups
- declaration of SIRM specific protocols, assays and annotation requirements for a given data modality.
- ISA roundtrip (write, reading, writing).
- Serialization to TAB and JSON
- Validation
   
 Stable Isotope Resolved Metabolomics Studies are a type of studies using MS and NMR acquisition techniques to decypher biochemical reactions using `tracer molecule`, i.e. molecules for which certain positions carry an isotope (e.g. 13C, 15N). Specific data acquisition and data processing techniques are required and dedicated software is used to make sense of the data. Software such as `IsoSolve` [1], `Ramid`[2](for primary processing of 13C mass isotopomer data obtained with GCMS) or `midcor`[3] (for natural abundance correction processes on13C mass isotopomers spectra), may be used to accomplish those tasks. The output of such tools are tables which may comply with a new specifications devised to better support the reporting of SIRM study results.
 
 
 - [1]. IsoSolve https://doi.org/10.1021/acs.analchem.1c01064
 - [2]. https://github.com/seliv55/ramid
 - [3]. https://github.com/seliv55/midcor


### Loading the ISA-API

In [1]:
!pip install git+https://github.com/isa-tools/isa-api.git@issue-511

  Cloning https://github.com/isa-tools/isa-api.git (to revision issue-511) to /private/var/folders/5n/rl6lqnks4rqb59pbtpvvntqw0000gr/T/pip-req-build-mzr7j2ry
  Running command git clone --filter=blob:none --quiet https://github.com/isa-tools/isa-api.git /private/var/folders/5n/rl6lqnks4rqb59pbtpvvntqw0000gr/T/pip-req-build-mzr7j2ry
  Running command git checkout -b issue-511 --track origin/issue-511
  Switched to a new branch 'issue-511'
  branch 'issue-511' set up to track 'origin/issue-511'.
  Resolved https://github.com/isa-tools/isa-api.git to commit 16ccc001fbdfaed073a6cb2f63d254c1b0b24a79
  Preparing metadata (setup.py) ... done


  Created wheel for isatools: filename=isatools-0.14.2-py3-none-any.whl size=2696947 sha256=ee443afda16dd3f9fa177a627370eb88ca515f6315e27eda76cc50de31b244fc
  Stored in directory: /private/var/folders/5n/rl6lqnks4rqb59pbtpvvntqw0000gr/T/pip-ephem-wheel-cache-qi3jve2_/wheels/ee/20/69/6f853ca26fa5c0d6a848162269e5640761b26197e3a5dde721
Successfully built isatools
  Attempting uninstall: isatools
    Found existing installation: isatools 0.12.0a0
ERROR: Exception:
Traceback (most recent call last):
  File "/Users/philippe/.pyenv/versions/3.9.0/lib/python3.9/site-packages/pip/_internal/cli/base_command.py", line 167, in exc_logging_wrapper
    status = run_func(*args)
  File "/Users/philippe/.pyenv/versions/3.9.0/lib/python3.9/site-packages/pip/_internal/cli/req_command.py", line 205, in wrapper
    return func(self, options, args)
  File "/Users/philippe/.pyenv/versions/3.9.0/lib/python3.9/site-packages/pip/_internal/commands/install.py", line 405, in run
    installed = install_given_reqs

In [2]:
!pip show isatools

Name: isatools
Version: 0.12.0a0
Summary: Metadata tracking tools help to manage an increasingly diverse set of life science, environmental and biomedical experiments
Home-page: https://github.com/ISA-tools/isa-api
Author: ISA Infrastructure Team
Author-email: isatools@googlegroups.com
License: UNKNOWN
Location: /Users/philippe/Documents/git/isa-api2/isa-api
Requires: beautifulsoup4, biopython, chardet, deepdiff, iso8601, jinja2, jsonschema, lxml, mzml2isa, networkx, numpy, pandas, progressbar2, PyYAML, requests
Required-by: 


In [3]:
import isatools


from isatools.model import (
    Comment,
    Investigation,
    Study,
    StudyFactor,
    FactorValue,
    OntologyAnnotation,
    Characteristic,
    OntologySource,
    Material,
    Sample,
    Source,
    Protocol,
    ProtocolParameter,
    ProtocolComponent,
    ParameterValue,
    Process,
    Publication,
    Person,
    Assay,
    DataFile,
    plink
)
import datetime
import os
from hashlib import md5, sha1, sha256, blake2b

def compute_hash(path, file, hash_func):
    """a subfunction generating the hash using hashlib functions

    :param path:
    :param file:
    :param hash_func:
    :return:
    """

    with open(os.path.join(path, file), "rb") as f:
        for byte_block in iter(lambda: f.read(4096), b""):
            hash_func.update(byte_block)
    return hash_func.hexdigest()


def update_checksum(path, isa_file_object: DataFile, checksum_type):
    """ a helper function to compute file checksum given a file path, an isa data file name and a type of algorithm

    :param path:
    :param isa_file_object:
    :param checksum_type: enum
    :return: isa_file_object:
    :raises ValueError: when the checksum is invalid
    """
    HASH_FUNCTIONS = {
        "md5": md5,
        "sha1": sha1,
        "sha256": sha256,
        "blake2": blake2b,
    }
    if checksum_type in HASH_FUNCTIONS.keys():
        hash_type = HASH_FUNCTIONS[checksum_type]()
        file_checksum = compute_hash(path, isa_file_object.filename, hash_type)
        isa_file_object.comments.append(Comment(name="checksum type", value=checksum_type))
    else:
        raise ValueError("Invalid checksum type")
    isa_file_object.comments.append(Comment(name="checksum", value=file_checksum))

    return isa_file_object

def md5_checksum(fname, path):
    hash_md5 = hashlib.md5()
    with open(os.path.join(path, fname), "rb") as f:
        for byte_block in iter(lambda: f.read(4096),b""):
            hash_md5.update(byte_block)
    return hash_md5.hexdigest()

def create_file():
    return filename


### Programmatic reporting of a 13C Stable Isotope Resolved Metabolomics (SIRM) study


#### Declaring the Ontologies and Vocabularies used in the ISA Study

In [4]:
investigation = Investigation()

chebi=OntologySource(name="CHEBI",
                     description="Chemical Entity of Biological Interest",
                     version="1.0",
                     file="https://www.example.org/CHEBI"
                    )
efo=OntologySource(name="EFO", description="Experimental Factor Ontology")
msio=OntologySource(name="MSIO", description="Metabolomics Standards Initiative Ontology")
obi = OntologySource(name='OBI', description="Ontology for Biomedical Investigations")
pato = OntologySource(name='PATO', description="Phenotype and Trait Ontology")
ncbitaxon = OntologySource(name="NCIBTaxon", description="NCBI Taxonomy")
uo = OntologySource(name="UO", description="Unit Ontology")

ontocomment = Comment(name="onto-test", value="onto-value")
ncbitaxon.comments.append(ontocomment)

investigation.ontology_source_references=[chebi, efo, obi, pato, ncbitaxon, msio, uo]

#### Declaring Units to be used at study or assay levels

In [5]:
mag_field_unit = OntologyAnnotation(term="Tesla", term_source=uo, term_accession="https://purl.org/")
mass_unit = OntologyAnnotation(term="mg", term_source=uo, term_accession="https://purl.org/")

#### Basic Study description: declaring Study Factor and and Study Design type

In [6]:
study = Study(filename="s_BH2023-study.txt")
study.identifier= "BH2023"
study.title = "[U-13C6]-D-glucose labeling experiment in MCF7 cancer cell line"
study.description = "Probing cancer pathways of MCF7 cell line using 13C stable isotope resolved metabolomics study using isotopologue distribution analysis with mass spectrometry and isotopomer analysis by 1D 1H NMR."
study.submission_date = "2021-08-15"
study.public_release_date = "2021-08-15"

# These EMBL-EBI Metabolights (MTBLS) related ISA Comments fields may be used for deposition to EMBL-EBI
SRA_comments = [
    {"name": "EMBL Broker Name", "value": "OXFORD"},
    {"name": "EMBL Center Name", "value": "OXFORD"},
    {"name": "EMBL Center Project Name", "value": "OXFORD"},
    {"name": "EMBL Lab Name", "value": "Oxford e-Research Centre"},
    {"name": "EMBL Submission Action", "value": "ADD"}
]

Funders_comments = [
    {"name": "Study Funding Agency", "value": ""},
    {"name": "Study Grant Number", "value": ""}    
]
for cmt in SRA_comments:
    sra_comment = Comment(name=cmt["name"], value=cmt["value"])
    study.comments.append(sra_comment)
    
for cmt in Funders_comments:
    funder_cmt = Comment(name=cmt["name"], value=cmt["value"])
    study.comments.append(funder_cmt)

# Adding a Study Design descriptor to the ISA Study object
intervention_design = OntologyAnnotation(term_source=obi)
intervention_design.term = "intervention design"
intervention_design.term_accession = "http://purl.obolibrary.org/obo/OBI_0000115"

study_design = OntologyAnnotation(term_source=msio)
study_design.term = "stable isotope resolved metabolomics study"
study_design.term_accession = "http://purl.obolibrary.org/obo/MSIO_0000096"

study.design_descriptors.append(intervention_design)
study.design_descriptors.append(study_design)


# Declaring the Study Factors
agent_ft_annot = OntologyAnnotation(term="chemical substance",
                                    term_accession="http://purl.obolibrary.org/obo/CHEBI_59999",
                                    term_source=chebi)
intensity_ft_annot = OntologyAnnotation(term="dose",
                                        term_accession="http://www.ebi.ac.uk/efo/EFO_0000428",
                                        term_source=efo)
duration_ft_annot = OntologyAnnotation(term="time",
                                       term_accession="http://purl.obolibrary.org/obo/PATO_0000165",
                                       term_source=pato)
study.factors = [
    StudyFactor(name="compound",factor_type=agent_ft_annot),
    StudyFactor(name="dose",factor_type=intensity_ft_annot),
    StudyFactor(name="duration",factor_type=duration_ft_annot)
]

# Associating the levels to each of the Study Factor.

agent_fvalue_annot = OntologyAnnotation(term="dioxygen", term_source=obi, term_accession="https://purl.org/")
intensity_fvalue_annot1 = OntologyAnnotation(term="high", term_source=obi, term_accession="https://purl.org/")
intensity_fvalue_annot2 =OntologyAnnotation(term="normal", term_source=obi, term_accession="https://purl.org/")
duration_fvalue_annot =OntologyAnnotation(term="hour", term_source=obi, term_accession="https://purl.org/")

fv1 = FactorValue(factor_name=study.factors[0], value=agent_fvalue_annot)
fv2 = FactorValue(factor_name=study.factors[1], value=intensity_fvalue_annot1)
fv3 = FactorValue(factor_name=study.factors[1], value=intensity_fvalue_annot2)
fv4 = FactorValue(factor_name=study.factors[2], value=duration_fvalue_annot)



#### Adding the publications associated to the study

In [7]:
status_annot_value = OntologyAnnotation(term="indexed in PubMed", term_source=obi, term_accession="https://purl.org/")

study.publications = [
    Publication(doi="10.1371/journal.pone.0000000",pubmed_id="36007233",
                title="Decyphering new cancer pathways with stable isotope resolved metabolomics in MCF7 cell lines",
                status=status_annot_value,
                author_list="Min,W. and Everest H"),
   
]

#### Adding the authors of the study

In [8]:

study.contacts = [
    Person(first_name="Weng", last_name="Min", affiliation="Beijing Institute of Metabolism", email="weng.min@bim.edu.cn",
           address="Prospect Street, Beijing, People's Republic of China",
           comments=[Comment(name="Study Person REF", value="")],
            roles=[OntologyAnnotation(term="principal investigator role"),
                   OntologyAnnotation(term="SRA Inform On Status"),
                   OntologyAnnotation(term="SRA Inform On Error")]
    ),
    Person(first_name="Hillary", last_name="Everest", affiliation="Centre for Cell Metabolism",
           address="CCM, Edinborough, United Kingdom",
           comments=[Comment(name="Study Person REF", value="")],
           roles=[OntologyAnnotation(term="principal investigator role")]
    )
]

#### Declaring all the protocols used in the ISA study. Note also the declaration of Protocol Parameters when needed.

In [9]:
study.protocols = [ 
    #Protocol #0
    Protocol(name="cell culture and isotopic labeling",
             description="SOP for growing MCF7 cells and incubating them with the tracer molecule",
             protocol_type=OntologyAnnotation(term="sample collection"),
             parameters=[
              ProtocolParameter(parameter_name=OntologyAnnotation(term="tracer molecule"))
             ]
            ),
    #Protocol #1
    Protocol(
        name="intracellular metabolite extraction",
        description="SOP for extracting metabolites from harvested cells",
        protocol_type=OntologyAnnotation(term="extraction")
    ),
    #Protocol #2
    Protocol(
        name="extracellular metabolite extraction",
        description="SOP for extracting metabolites from cell culture supernatant",
        protocol_type=OntologyAnnotation(term="extraction")
    ),
    #Protocol #3
    Protocol(
        name="liquid chromatography mass spectrometry",
        description="SOP for LC-MS data acquisition",
        protocol_type=OntologyAnnotation(term="mass spectrometry"),
        parameters=[
            ProtocolParameter(parameter_name=OntologyAnnotation(term="chromatography column")),
            ProtocolParameter(parameter_name=OntologyAnnotation(term="mass spectrometry instrument")),
            ProtocolParameter(parameter_name=OntologyAnnotation(term="mass analyzer"))
        ]
    ),
    #Protocol #4
    Protocol(
        name="1D 13C NMR spectroscopy for isotopomer analysis",
        description="SOP for 1D 13C NMR data acquisition for isotopomer analysis",
        protocol_type=OntologyAnnotation(term="NMR spectroscopy"),
        parameters=[
            ProtocolParameter(parameter_name=OntologyAnnotation(term="magnetic field strength")),
            ProtocolParameter(parameter_name=OntologyAnnotation(term="nmr tube")),
            ProtocolParameter(parameter_name=OntologyAnnotation(term="pulse sequence"))
        ]
    ),
    #Protocol #5
    Protocol(
        name="1D 13C NMR spectroscopy for metabolite profiling",
        description="SOP for 1D 13C NMR data acquisition for metabolite profiling",
        protocol_type=OntologyAnnotation(term="NMR spectroscopy"),
        parameters=[
            ProtocolParameter(parameter_name=OntologyAnnotation(term="magnetic field strength")),
            ProtocolParameter(parameter_name=OntologyAnnotation(term="nmr tube")),
            ProtocolParameter(parameter_name=OntologyAnnotation(term="pulse sequence"))
        ]
    ),
    #Protocol #6
    Protocol(
        name="MS metabolite identification",
        description="SOP for MS signal processing and metabolite and isotopologue identification",
        protocol_type=OntologyAnnotation(term="metabolite identification"),
        parameters=[
            ProtocolParameter(parameter_name=OntologyAnnotation(term="ms software"))
        ]
    ),
    #Protocol #7
    Protocol(
        name="NMR metabolite identification",
        description="SOP for NMR signal processing and metabolite and isotopomer identification",
        uri="https://doi.org/10.1021/acs.analchem.1c01064",
        protocol_type=OntologyAnnotation(term="data transformation"),
        parameters=[
            ProtocolParameter(parameter_name=OntologyAnnotation(term="nmr software"))
        ]
    ),
    
    #Protocol #8
    Protocol(
        name="mRNA extraction",
        description="procedure for isolating messenger RNA for transcriptomics analysis",
        uri="",
        protocol_type=OntologyAnnotation(term="material separation")
    ),
    
    #Protocol #9
    Protocol(
        name="gDNA extraction",
        description="procedure for isolating genomic DNA for copy number variation analysis",
        uri="",
        protocol_type=OntologyAnnotation(term="material separation")
    ),
    
    #Protocol #10
    Protocol(
        name="gDNA library preparation",
        description="procedure for isolating genoic DNA for copy number variation analysis",
        uri="",
        protocol_type=OntologyAnnotation(term="library construction"),
        parameters=[
            ProtocolParameter(parameter_name=OntologyAnnotation(term="library strategy")),
            ProtocolParameter(parameter_name=OntologyAnnotation(term="library selection")),
            ProtocolParameter(parameter_name=OntologyAnnotation(term="library source")),
            ProtocolParameter(parameter_name=OntologyAnnotation(term="library orientation"))          
        ]
    ),
    
    #Protocol #11
    Protocol(
        name="mRNA library preparation",
        description="procedure for isolating genoic DNA for gene expression analysis",
        uri="",
        protocol_type=OntologyAnnotation(term="library construction"),
        parameters=[
            ProtocolParameter(parameter_name=OntologyAnnotation(term="library strategy")),
            ProtocolParameter(parameter_name=OntologyAnnotation(term="library selection")),
            ProtocolParameter(parameter_name=OntologyAnnotation(term="library source")),
            ProtocolParameter(parameter_name=OntologyAnnotation(term="library orientation"))          
        ]
    ),
    
    #Protocol #12
    Protocol(
        name="nucleic acid sequencing",
        description="SOP for nucleic acid sequencing",
        uri="",
        protocol_type=OntologyAnnotation(term="nucleic acid sequencing"),
        parameters=[
            ProtocolParameter(parameter_name=OntologyAnnotation(term="sequencing instrument"))
        ]
    ),
    
    #Protocol #13
    Protocol(
        name="transcription analysis",
        description="SOP for transcriptomics analysis",
        uri="",
        protocol_type=OntologyAnnotation(term="data transformation"),
        parameters=[
            ProtocolParameter(parameter_name=OntologyAnnotation(term="sequence analysis software"))
        ]
    ),
    
    #Protocol #14
    Protocol(
        name="CNV analysis",
        description="SOP for CNV ",
        uri="",
        protocol_type=OntologyAnnotation(term="data transformation"),
        parameters=[
            ProtocolParameter(parameter_name=OntologyAnnotation(term="variant calling software"))
        ]
    )
    
]

### Now, creating ISA Source and Sample objects and building the ISA study table


In this fictional study, we assume the following underlying experimental setup:

- Human MCF-7 breast cancer cell line will be grown in 2 distinct conditions, namely "normal concentration of dioxygen for 72 hours" and "high concentration of dioxygen for 72 hours".
- Each cell culture batch will be grown in the presence of 80% [1-13C1]-D-glucose + 20% [U-13C6]-D-glucose tracer molecules
- For each cell culture, 4 samples will be collected for characterisation.
- 3 assay modalities will be used on each of the collected samples, namely:
    - isotopologue distribution analysis using LC-MS
    - isotopomer analysis using 1D 13C NMR spectrometry with 3 distinct pulse sequences (HSQC, ZQF-TOCSY, HCNA, and HACO-DIPSY)
    - metabolite profiling using 1D 13C NMR spectrometry with one pulse sequence (CPMG)
- Data analysis for each data modality will be performed with dedicated data analysis protocols
- Integrative analysis (cutting accross results coming from each assay modality) will be performed using a dedicated workflow.

#### Let's start by building the ISA Source and ISA Samples reflecting the experimental plan:

In [10]:
# Creating the ISA Source Materials
study.sources = [Source(name="culture-1"), Source(name="culture-2")]

src_characteristic_biosamplexref = Characteristic(category=OntologyAnnotation(term="namespace:biosample:src"),
                                     value=OntologyAnnotation(term="SRC:" ,
                                                              term_source=obi, term_accession="https://purl.org/"))

characteristic_organism = Characteristic(category=OntologyAnnotation(term="Organism"),
                                     value=OntologyAnnotation(term="Homo sapiens",
                                                              term_source=ncbitaxon,
                                                              term_accession="http://purl.obolibrary.org/obo/NCBITaxon_9606"))

characteristic_cell = Characteristic(category=OntologyAnnotation(term="cell line"),
                                     value=OntologyAnnotation(term="MCF-7", term_source=obi, term_accession="https://purl.org/"))



study.characteristic_categories.append(src_characteristic_biosamplexref.category)
study.characteristic_categories.append(characteristic_organism.category)
study.characteristic_categories.append(characteristic_cell.category)


for i in range(len(study.sources)):  
    study.sources[i].characteristics.append(src_characteristic_biosamplexref)
    study.sources[i].characteristics.append(characteristic_organism)
    study.sources[i].characteristics.append(characteristic_cell)
    

# Note how the treatment groups are defined as sets of factor values attached to the ISA.Sample object
treatment_1 = [fv1,fv2,fv4]
treatment_2 = [fv1,fv3,fv4]


# Ensuring the Tracer Molecule(s) used for the SIRM study is properly reported
tracer_mol_C = ParameterValue(category=ProtocolParameter(parameter_name=OntologyAnnotation(term="tracer molecule",
                                                                                           term_source="",
                                                                                           term_accession="")),
                            value=OntologyAnnotation(term="80% [1-13C1]-D-glucose + 20% [U-13C6]-D-glucose",
                                                     term_source=chebi,
                                                     term_accession="https://purl.org/chebi_1212"))


tracers = [tracer_mol_C]

# the number of samples collected from each culture condition
replicates = 4
# Now creating a Process showing a `Protocol Application` using Source as input and producing Sample as output.

for k in range(replicates):
    
    smp_characteristics_biosamplexref = Characteristic(category=OntologyAnnotation(term="namespace:biosample:smp"),
                                                       value=OntologyAnnotation(term=("SAME:" + str(k)), term_source=obi, term_accession="https://purl.org/"))
    
    study.characteristic_categories.append(smp_characteristics_biosamplexref.category)
    
    study.samples.append(Sample(name=(study.sources[0].name + "-sample-" + str(k)),
                                characteristics=[smp_characteristics_biosamplexref],
                                factor_values=treatment_1))
    
    study.samples.append(Sample(name=(study.sources[1].name + "-sample-" + str(k)), 
                                characteristics=[smp_characteristics_biosamplexref],
                                factor_values=treatment_2))

sample_collection_mbx = Process(name="sample-collection-process-mbx",
        executes_protocol=study.protocols[0], # a sample collection
        inputs=[study.sources[0]],
        outputs=[study.samples[0],study.samples[2],study.samples[4],study.samples[6]],
        parameter_values= [tracer_mol_C])

sample_collection_gtx = Process(name="sample-collection-process-gtx",
        executes_protocol=study.protocols[0], # a sample collection
        inputs=[study.sources[1]],
        outputs=[study.samples[1],study.samples[3],study.samples[5],study.samples[7]],
        parameter_values= [tracer_mol_C])

study.process_sequence.append(sample_collection_mbx)

study.process_sequence.append(sample_collection_gtx)

study.units = []
                          
# Now appending the ISA Study object to the ISA Investigation object    
investigation.studies = [study]



### Now, creating the ISA objects needed to represent assays and raw data acquisition.

#### Let's start by declaring the 5 modalities as 5 ISA Assays.

In [11]:
#Starting by declaring the 2 types of assays used in BII-S-3 as coded with ISAcreator tool
assay = Assay(filename="a_"+ study.identifier + "-isotopologue-ms-assay.txt")
assay.measurement_type = OntologyAnnotation(term="isotopologue distribution analysis",term_accession="http://purl.obolibrary.org/obo/msio.owl#mass_isotopologue_distribution_analysis", term_source=msio)
assay.technology_type = OntologyAnnotation(term="mass spectrometry", term_accession="http://purl.obolibrary.org/obo/CHMO_0000470", term_source=msio)
assay.comments.append(Comment(name="target repository", value="metabolights"))

assay_nmr_topo = Assay(filename="a_"+ study.identifier + "-isotopomer-nmr-assay.txt")
assay_nmr_topo.measurement_type = OntologyAnnotation(term="isotopomer analysis",term_accession="http://purl.obolibrary.org/obo/msio.owl#isotopomer_analysis", term_source=msio)
assay_nmr_topo.technology_type = OntologyAnnotation(term="NMR spectroscopy",term_accession="http://purl.obolibrary.org/obo/CHMO_0000591", term_source=msio)
assay_nmr_topo.comments.append(Comment(name="target repository", value="metabolights"))
#
assay_nmr_metpro = Assay(filename="a_"+ study.identifier + "-metabolite-profiling-nmr-assay.txt")
assay_nmr_metpro.measurement_type = OntologyAnnotation(term="metabolite profiling",term_accession="http://purl.obolibrary.org/obo/MSIO_0000101", term_source=msio)
assay_nmr_metpro.technology_type = OntologyAnnotation(term="NMR spectroscopy",term_accession="http://purl.obolibrary.org/obo/CHMO_0000591", term_source=msio)
assay_nmr_metpro.comments.append(Comment(name="target repository", value="metabolights"))

assay_cnv_seq = Assay(filename="a_"+ study.identifier + "-cnv_seq-assay.txt")
assay_cnv_seq.measurement_type = OntologyAnnotation(term="copy number variation profiling",term_accession="https://purl.org", term_source=msio)
assay_cnv_seq.technology_type = OntologyAnnotation(term="nucleotide sequencing",term_accession="https://purl.org", term_source=msio)
assay_cnv_seq.comments.append(Comment(name="target repository", value="ega"))

assay_rna_seq = Assay(filename="a_"+ study.identifier + "-rna-seq-assay.txt")
assay_rna_seq.measurement_type = OntologyAnnotation(term="transcription profiling", term_accession="https://purl.org", term_source=msio)
assay_rna_seq.technology_type = OntologyAnnotation(term="nucleotide sequencing", term_accession="https://purl.org", term_source=msio)
assay_rna_seq.comments.append(Comment(name="target repository", value="arrayexpress"))




**Warning**

- The current release of ISA-API throws an error if Assay `technology type` OntologyAnnotation.term is left empty
- The coming release 10.13 will address this issue.


### The mass isotopologue distribution analysis assay using MS acquisitions:

In [12]:
main_path = "./output/ISA-BH2023-ALL/"
data_path = "./output/"

In [13]:
# ms_sw = ParameterValue(category=ProtocolParameter(parameter_name=OntologyAnnotation(term="ms software")),
#                        value=OntologyAnnotation(term="IsoSolve", term_source=obi, term_accession="https://purl.org/"))


# ms_derivedDF = DataFile(filename="isotopologue-distribution-analysis.txt", label="Derived Data File")

# f=open(os.path.join(main_path,"DERIVED_FILES/","isotopologue-distribution-analysis.txt"),"w+")
# f.write("isotopologue-distribution-analysis.txt")
# f.close

# ms_da_process = Process(name="MS-DT-ident",
#                         executes_protocol=study.protocols[6],
#                         parameter_values=[ms_sw],
#                         outputs=[ms_derivedDF])

# assay.data_files.append(ms_derivedDF)

# for i, sample in enumerate(study.samples):



#     # extraction process takes as input a sample, and produces an extract material as output

#     char_ext = Characteristic(category=OntologyAnnotation(term="Material Type"),
#                                      value=OntologyAnnotation(term="pellet", term_source=obi, term_accession="https://purl.org/"))

   
#     char_ext1 = Characteristic(category=OntologyAnnotation(term="quantity"),
#                                value=40, 
#                                unit=mass_unit
#                               )
#     ms_material = Material(name="extract-ms-{}".format(i))
#     ms_material.type = "Extract Name"
#     ms_material.characteristics.append(char_ext)
#     ms_material.characteristics.append(char_ext1)
#     # create an extraction process that executes the extraction protocol

    
#     extraction_process = Process(
#         name="extract-process-{}".format(i),
#         executes_protocol=study.protocols[1],
#         inputs=[sample],
#         outputs=[ms_material]
#     )

#     # create a ms acquisition process that executes the ms acquisition protocol
#     column = ParameterValue(category=ProtocolParameter(parameter_name=OntologyAnnotation(term="chromatography column")),
#                             value=OntologyAnnotation(term="Agilent C18 TTX", term_source=obi, term_accession="https://purl.org/"))
#     ms_inst = ParameterValue(category=ProtocolParameter(parameter_name=OntologyAnnotation(term="mass spectrometry instrument")),
#                             value=OntologyAnnotation(term="Agilent QTOF XL", term_source=obi, term_accession="https://purl.org/"))
#     ms_anlzr = ParameterValue(category=ProtocolParameter(parameter_name=OntologyAnnotation(term="mass analyzer")),
#                             value=OntologyAnnotation(term="Agilent MassDiscovery", term_source=obi, term_accession="https://purl.org/"))

#     datafile = DataFile(
#         filename="ms-data-{}.mzml".format(i),
#         label="Raw Spectral Data File"
#     )
#     f=open(os.path.join(main_path, "RAW_FILES/","ms-data-{}.mzml".format(i)),"w+")
#     f.write("ms-data-{}.mzml".format(i))
#     f.close
#     data_comment = Comment(name="data_comment",value="data_value")
#     datafile.comments.append(data_comment)
                                
#     isotopologue_process = Process(
#         name="assay-name-ms-{}".format(i),
#         executes_protocol=study.protocols[3],
#         parameter_values=[column, ms_inst, ms_anlzr],     
#         inputs=[extraction_process.outputs[0]],
#         outputs=[datafile]
#     )

#     # ms acquisition process usually has an output mzml data file
#     ms_da_process.inputs.append(datafile) 

#     # Ensure Processes are linked forward and backward. plink(from_process, to_process) is a function to set
#     # these links for you. It is found in the isatools.model package

#     assay.samples.append(sample)
#     assay.other_material.append(ms_material)
#     assay.data_files.append(datafile)

#     assay.process_sequence.append(extraction_process)
#     assay.process_sequence.append(isotopologue_process)
#     assay.process_sequence.append(ms_da_process)
#     # create an extraction process that executes the extraction protocol

#     # plink(aliquoting_process, sequencing_process)
#     plink(sample_collection_mbx, extraction_process)
#     plink(extraction_process, isotopologue_process)
#     plink(isotopologue_process, ms_da_process)
#     # make sure the extract, data file, and the processes are attached to the assay

# assay.characteristic_categories.append(char_ext.category)
# assay.characteristic_categories.append(char_ext1.category)
# assay.units.append(mass_unit)

**NOTE**
make sure to used `ISA API plink function` to connects the protocols in a chain.

### The mass isotopomer distribution analysis assay using 1D 13C NMR acquisitions:

In [14]:
# nmr_sw = ParameterValue(category=ProtocolParameter(parameter_name=OntologyAnnotation(term="nmr software")),
#                                value=OntologyAnnotation(term="https://pypi.org/project/IsoSolve", term_source=obi, term_accession="https://purl.org/"))

# nmr_topo_DDF = DataFile(filename="isotopomer-analysis.txt", label="Derived Data File")


# f=open(os.path.join(main_path, "DERIVED_FILES/","isotopomer-analysis.txt"),"w+")
# f.write("isotopomer-analysis.txt")
# f.close


# nmr_topo_da_process = Process(
#     name = "NMR-TOPO-DT-ident",
#     executes_protocol=study.protocols[7],
#     parameter_values=[nmr_sw],
#     outputs=[nmr_topo_DDF]
# )

# assay_nmr_topo.data_files.append(nmr_topo_DDF)

# for i, sample in enumerate(study.samples):
    
#     # extraction process takes as input a sample, and produces an extract material as output
#     material_nmr = Material(
#         name="extract-nmr-topo-{}".format(i),
#         type_="Extract Name"
#     )
    
#     extraction_process_nmr = Process(
#         name="extract-process-{}".format(i),
#         executes_protocol=study.protocols[1],
#         inputs=[sample],
#         outputs=[material_nmr]
#     )

#     # create a nmr acquisition process that executes the nmr protocol
#     magnet = ParameterValue(category=ProtocolParameter(parameter_name=OntologyAnnotation(term="magnetic field strength")),
#                             value=6, unit=mag_field_unit)
#     tube = ParameterValue(category=ProtocolParameter(parameter_name=OntologyAnnotation(term="nmr tube")),
#                             value=OntologyAnnotation(term="Brucker 14 mm Oscar", term_source=obi, term_accession="https://purl.org/"))
#     pulse_a = ParameterValue(category=ProtocolParameter(parameter_name=OntologyAnnotation(term="pulse sequence")),
#                             value=OntologyAnnotation(term="HSQC", term_source=obi, term_accession="https://purl.org/"))
#     pulse_b = ParameterValue(category=ProtocolParameter(parameter_name=OntologyAnnotation(term="pulse sequence")),
#                             value=OntologyAnnotation(term="ZQF-TOCSY", term_source=obi, term_accession="https://purl.org/"))
#     pulse_c = ParameterValue(category=ProtocolParameter(parameter_name=OntologyAnnotation(term="pulse sequence")),
#                             value=OntologyAnnotation(term="HNCA", term_source=obi, term_accession="https://purl.org/"))
#     pulse_d = ParameterValue(category=ProtocolParameter(parameter_name=OntologyAnnotation(term="pulse sequence")),
#                             value=OntologyAnnotation(term="HACO-DIPSY", term_source=obi, term_accession="https://purl.org/"))

#     pulses=[pulse_a,pulse_b,pulse_c,pulse_d]

#     for j in range(len(pulses)):

#         # Sequencing process usually has an output data file
#         datafile_nmr = DataFile(filename="nmr-data-topo"+  pulses[j].value.term +"-{}.nmrml".format(i+1), label="Free Induction Decay Data File")
#         f=open(os.path.join(main_path,"RAW_FILES/","nmr-data-topo"+  pulses[j].value.term +"-{}.nmrml".format(i+1)),"w+")
#         f.write("nmr-data-topo"+  pulses[j].value.term +"-{}.nmrml".format(i+1))
#         f.close
        
#         isotopomer_process = Process(
#             name = "assay-name-nmr-topo-"+ pulses[j].value.term +"-{}".format(i+1),
#             executes_protocol=study.protocols[4],
#             parameter_values=[magnet,tube,pulses[j]],
#             inputs=[extraction_process_nmr.outputs[0]],
#             outputs=[datafile_nmr]
#         )

#         # Ensure Processes are linked forward and backward. plink(from_process, to_process) is a function to set
#         # these links for you. It is found in the isatools.model package

#         assay_nmr_topo.samples.append(sample)
#         assay_nmr_topo.other_material.append(material_nmr)
#         assay_nmr_topo.data_files.append(datafile_nmr)

#         assay_nmr_topo.process_sequence.append(extraction_process_nmr)
#         assay_nmr_topo.process_sequence.append(isotopomer_process)
#         assay_nmr_topo.process_sequence.append(nmr_topo_da_process)
        
#         plink(sample_collection_mbx, extraction_process_nmr)
#         plink(extraction_process_nmr, isotopomer_process)
#         plink(isotopomer_process, nmr_topo_da_process)
#         # make sure the extract, data file, and the processes are attached to the assay

    
    
# assay_nmr_topo.units.append(mag_field_unit)

### Conventional Metabolite Profiling  using dedicated 1D 13C NMR acquisitions:

In [15]:
nmr_sw = ParameterValue(category=ProtocolParameter(parameter_name=OntologyAnnotation(term="nmr software")),
                                value=OntologyAnnotation(term="Batman", term_source=obi, term_accession="https://purl.org/"))

nmr_derivedDF = DataFile(filename="metpro-analysis.txt", label="Derived Spectral Data File")
f=open(os.path.join(main_path, "DERIVED_FILES/","metpro-analysis.txt"),"w+")
f.write("metpro-analysis.txt")
f.close

nmr_da_process = Process(
    name = "NMR-metpro-DT-ident",
    executes_protocol=study.protocols[7],
    parameter_values=[nmr_sw],
    outputs=[nmr_derivedDF]
)

assay_nmr_metpro.data_files.append(nmr_derivedDF)

for i, sample in enumerate(study.samples):
    
#    extraction process takes as input a sample, and produces an extract material as output

    material_nmr_metpro = Material(name="extract-nmr-metpro-{}".format(i),
                                   type_="Extract Name")
    
    extraction_process_nmr_metpro = Process(
        name="extract-process-{}".format(i),
        executes_protocol=study.protocols[1], 
        inputs=[sample],
        outputs=[material_nmr_metpro]
    )
    
    
    # create a nmr acquisition process that executes the nmr protocol
    magnet = ParameterValue(category=ProtocolParameter(parameter_name=OntologyAnnotation(term="magnetic field strength")),
                            value=6.5,
                            unit=mag_field_unit)
    tube = ParameterValue(category=ProtocolParameter(parameter_name=OntologyAnnotation(term="nmr tube")),
                            value=OntologyAnnotation(term="Brucker 14 mm Oscar", term_source=obi, term_accession="https://purl.org/"))
    pulse_a = ParameterValue(category=ProtocolParameter(parameter_name=OntologyAnnotation(term="pulse sequence")),
                            value=OntologyAnnotation(term="CPMG", term_source=obi, term_accession="https://purl.org/"))

#     pulses=[pulse_a]

#     for j in range(len(pulses)):

    metpro_process = Process(executes_protocol=study.protocols[5],parameter_values=[magnet,tube,pulse_a])
    metpro_process.name = "assay-name-nmr-metpro-"+ pulse_a.value.term +"-{}".format(i+1)
    metpro_process.inputs.append(extraction_process_nmr_metpro.outputs[0])

        # a Data acquisition process usually has an output data file

    datafile_nmr_metpro = DataFile(filename="nmr-data-metpro-"+pulse_a.value.term +"-{}.nmrml".format(i+1), label="Free Induction Decay Data File")
    f=open(os.path.join(main_path,"RAW_FILES/","nmr-data-metpro-"+ pulse_a.value.term +"-{}.nmrml".format(i+1)),"w+")
    f.write("nmr-data-metpro-"+ pulse_a.value.term +"-{}.nmrml".format(i+1))
    f.close

    metpro_process.outputs.append(datafile_nmr_metpro)
    nmr_da_process.inputs.append(datafile_nmr_metpro)

        # Ensure Processes are linked forward and backward. plink(from_process, to_process) is a function to set
        # these links for you. It is found in the isatools.model package

    assay_nmr_metpro.samples.append(sample)
    assay_nmr_metpro.other_material.append(material_nmr_metpro)
    assay_nmr_metpro.data_files.append(datafile_nmr_metpro)

    assay_nmr_metpro.process_sequence.append(extraction_process_nmr_metpro)
    assay_nmr_metpro.process_sequence.append(metpro_process)
    assay_nmr_metpro.process_sequence.append(nmr_da_process)        

    plink(sample_collection_mbx, extraction_process_nmr_metpro)
    plink(extraction_process_nmr_metpro, metpro_process)
    plink(metpro_process, nmr_da_process)
# make sure the extract, data file, and the processes are attached to the assay


    
assay_nmr_metpro.units.append(mag_field_unit)


### Transcriptomics profiling using sequencing assay:

In [16]:
#TODO: this is static: take it out of the for loop
char_ext_rna_seq = Characteristic(category=OntologyAnnotation(term="Stuff Type"),
                                  value=OntologyAnnotation(term="mRNA", term_source=obi, term_accession="https://purl.org/"))

#TODO: this is static: take it out of the for loop
rna_strat = ParameterValue(category=ProtocolParameter(parameter_name=OntologyAnnotation(term="library strategy")),
                        value=OntologyAnnotation(term="RNA-SEQ", term_source=obi, term_accession="https://purl.org/"))
rna_sel = ParameterValue(category=ProtocolParameter(parameter_name=OntologyAnnotation(term="library selection")),
                        value=OntologyAnnotation(term="OTHER", term_source=obi, term_accession="https://purl.org/"))
rna_src = ParameterValue(category=ProtocolParameter(parameter_name=OntologyAnnotation(term="library source")),
                        value=OntologyAnnotation(term="TRANSCRIPTOMICS", term_source=obi, term_accession="https://purl.org/"))
rna_ori = ParameterValue(category=ProtocolParameter(parameter_name=OntologyAnnotation(term="library orientation")),
                        value=OntologyAnnotation(term="SINGLE", term_source=obi, term_accession="https://purl.org/"))


rna_label = Characteristic(category=OntologyAnnotation(term="Label"), value=OntologyAnnotation(term="AAAAAAAAAA", term_source=obi, term_accession="https://purl.org/"))

seq_instrument = ParameterValue(category=ProtocolParameter(parameter_name=OntologyAnnotation(term="sequencing instrument")),
                                    value=OntologyAnnotation(term="Illumina MiSeq", term_source=obi, term_accession="https://purl.org/"))

rna_sw = ParameterValue(category=ProtocolParameter(parameter_name=OntologyAnnotation(term="sequence analysis software")),
                            value=OntologyAnnotation(term="DESeq2", term_source=obi, term_accession="https://purl.org/"))

for i, sample in enumerate(study.samples):

    # extraction process takes as input a sample, and produces an extract material as output
    
    material_rna_seq = Material(name="extract-rna-seq-{}".format(i))
    material_rna_seq.type = "Extract Name"
    material_rna_seq.characteristics.append(char_ext_rna_seq)
    # print(char_ext_rna_seq.to_dict())
    
    # create an extraction process that executes the extraction protocol

    extraction_process_rna_seq = Process(
        name="extract-process-rna-seq-{}".format(i),
        executes_protocol=study.protocols[8],
        inputs=[sample],
        outputs=[material_rna_seq]
    )
    

    # create a library contruction process that executes the gDNA library construction protocol


    rna_library = Material(name="rna-library-name-{}".format(i))
    rna_library.type = "Labeled Extract Name"
    rna_library.characteristics.append(rna_label)
    
    rna_lib_process = Process(
        name = "rna-library-name-{}".format(i),
        executes_protocol=study.protocols[11],
        parameter_values=[rna_strat,rna_sel, rna_src, rna_ori],
        inputs=[extraction_process_rna_seq.outputs[0]],
        outputs=[rna_library]
    )

    # rna seq acquisition process usually has an output fastq data file
    rna_datafile = DataFile(filename="rna-seq-data-{}.fastq".format(i), label="Raw Data File")
    f=open(os.path.join(main_path, "rna-seq-data-{}.fastq".format(i)),"w+")
    f.write("rna-seq-data-{}.fastq".format(i))
    # f.close


    updated_rna_datafile = update_checksum(main_path, rna_datafile, "md5")

    rna_data_comment = Comment(name="export",value="yes")
    # rna_data_comment1 = Comment(name="checksum", value=md5)
    # rna_data_comment2 = Comment(name="checksum type", value="MD5")

    updated_rna_datafile.comments.append(rna_data_comment)
    # rna_datafile.comments.append(rna_data_comment1)
    # rna_datafile.comments.append(rna_data_comment2)
    #
    
    # create a sequencing process that executes the sequencing protoco
    rna_seq_process = Process(
        name = "assay-name-rna-seq-{}".format(i),
        executes_protocol=study.protocols[12],
        parameter_values=[seq_instrument],
        inputs=[rna_lib_process.outputs[0]],
        outputs=[updated_rna_datafile]
    )

    # Ensure Processes are linked forward and backward. plink(from_process, to_process) is a function to set
    # these links for you. It is found in the isatools.model package

    assay_rna_seq.samples.append(sample)
    assay_rna_seq.other_material.append(material_rna_seq)
    assay_rna_seq.other_material.append(rna_library)
    assay_rna_seq.data_files.append(updated_rna_datafile)

    
    rnaseq_drvdf = DataFile(filename="rna-seq-DEA.txt", label="Derived Data File")
    dvf=open(os.path.join(main_path,"rna-seq-DEA.txt"),"w+")
    dvf.write("rna-seq-DEA.txt")
    # dvf.close
    

    rna_drvdata_comment  = Comment(name="export", value="yes")
    updated_rnaseq_drvdf = update_checksum(main_path, rnaseq_drvdf, "md5")
    updated_rnaseq_drvdf.comments.append(rna_drvdata_comment)

    
    rna_da_process = Process(
        name = "RNASEQ-DT",
        executes_protocol=study.protocols[13],
        parameter_values=[rna_sw],
        inputs=[rna_datafile],
        outputs=[updated_rnaseq_drvdf]
    )
    
    assay_rna_seq.process_sequence.append(extraction_process_rna_seq)
    assay_rna_seq.process_sequence.append(rna_lib_process)
    assay_rna_seq.process_sequence.append(rna_seq_process)
    assay_rna_seq.process_sequence.append(rna_da_process)
    
    plink(sample_collection_gtx, extraction_process_rna_seq)
    plink(extraction_process_rna_seq, rna_lib_process)
    plink(rna_lib_process, rna_seq_process)
    plink(rna_seq_process, rna_da_process)

    
assay_rna_seq.characteristic_categories.append(char_ext_rna_seq.category)


### Copy Number Variation profiling using sequencing:

In [17]:
char_ext_cnv_seq = Characteristic(category=OntologyAnnotation(term="Stuff Type", term_source="", term_accession=""),
                                     value=OntologyAnnotation(term="gDNA", term_source=obi, term_accession="https://purl.org/OBOfoundry/obi:123414"))

# create a library contruction process that executes the gDNA library construction protocol
cnv_strat = ParameterValue(category=ProtocolParameter(parameter_name=OntologyAnnotation(term="library strategy")),
                        value=OntologyAnnotation(term="WGS", term_source=obi, term_accession="https://purl.org/"))
cnv_sel = ParameterValue(category=ProtocolParameter(parameter_name=OntologyAnnotation(term="library selection")),
                        value=OntologyAnnotation(term="OTHER", term_source=obi, term_accession="https://purl.org/"))
cnv_src = ParameterValue(category=ProtocolParameter(parameter_name=OntologyAnnotation(term="library source")),
                        value=OntologyAnnotation(term="GENOMICS", term_source=obi, term_accession="https://purl.org/"))
cnv_ori = ParameterValue(category=ProtocolParameter(parameter_name=OntologyAnnotation(term="library orientation")),
                        value=OntologyAnnotation(term="SINGLE", term_source=obi, term_accession="https://purl.org/"))

cnv_label = Characteristic(category=OntologyAnnotation(term="Label") , value=OntologyAnnotation(term="Not Applicable", term_source=obi, term_accession="https://purl.org/"))

seq_instrument = ParameterValue(category=ProtocolParameter(parameter_name=OntologyAnnotation(term="sequencing instrument")),
                            value=OntologyAnnotation(term="AB SOLiD 5500xl", term_source=obi, term_accession="https://purl.org/")) 

cnv_sw = ParameterValue(category=ProtocolParameter(parameter_name=OntologyAnnotation(term="variant calling software")),
                            value=OntologyAnnotation(term="VCF caller", term_source=obi, term_accession="https://purl.org/"))

for i, sample in enumerate(study.samples): 

    # extraction process takes as input a sample, and produces an extract material as output
        
    material_cnv_seq = Material(name="extract-cnv-seq-{}".format(i))
    material_cnv_seq.type = "Extract Name"
    material_cnv_seq.characteristics.append(char_ext_cnv_seq)
    print(material_cnv_seq.characteristics)
    
     # create an extraction process that executes the extraction protocol
    extraction_process_cnv_seq = Process(
        name="extract-process-cnv-seq-{}".format(i),
        executes_protocol=study.protocols[9],
        inputs=[sample],
        outputs=[material_cnv_seq]
    )

    cnv_library = Material(name="cnv-library-name-{}".format(i))
    cnv_library.type = "Labeled Extract Name"
    cnv_library.characteristics.append(cnv_label)
    
    cnv_lib_process = Process(
        name = "cnv-library-name-{}".format(i),
        executes_protocol=study.protocols[10],
        parameter_values=[cnv_strat,cnv_sel, cnv_src, cnv_ori],
        inputs=[extraction_process_cnv_seq.outputs[0]],
        outputs=[cnv_library]
    )


    # cnv seq acquisition process usually has an output fastq data file

    cnv_datafile = DataFile(filename="cnv-seq-data-{}.fastq".format(i), label="Raw Data File")
    f=open(os.path.join(main_path,"cnv-seq-data-{}.fastq".format(i)), "w+")

    cnv_data_comment = Comment(name="export", value="yes")
    updated_cnv_datafile = update_checksum(main_path, cnv_datafile, "md5")
    updated_cnv_datafile.comments.append(cnv_data_comment)
    
    
    # create a sequencing process that executes the sequencing protocol
    cnv_seq_process = Process(
        name = "assay-name-cnv-seq-{}".format(i),
        executes_protocol=study.protocols[12],
        parameter_values=[seq_instrument],
        inputs=[cnv_lib_process.outputs[0]],
        outputs=[updated_cnv_datafile]
    )

    # Ensure Processes are linked forward and backward. plink(from_process, to_process) is a function to set
    # these links for you. It is found in the isatools.model package

    assay_cnv_seq.samples.append(sample)
    assay_cnv_seq.other_material.append(material_cnv_seq)
    assay_cnv_seq.other_material.append(cnv_library)
    assay_cnv_seq.data_files.append(updated_cnv_datafile)


    cnvseq_drvdf = DataFile(filename="cnv-seq-derived-data.vcf", label="Derived Data File")
    dvf=open(os.path.join(main_path,"cnv-seq-derived-data.vcf"),"w+")
    dvf.write("cnv-seq-derived-datav.vcf")
    # dvf.close
    
    
    # cnvseq_drvdf = DataFile(filename="cnv-seq-data-{}.vcf".format(i), label="Derived Data File")
    # dvf=open(os.path.join(main_path,"cnv-seq-data-{}.vcf".format(i)),"w+")
    # dvf.write("cnv-seq-data-{}.vcf".format(i))
    # dvf.close
    

    cnv_drvdata_comment = Comment(name="export",value="yes")    
    updated_cnvseq_drvdf = update_checksum(main_path, cnvseq_drvdf, "md5")
    updated_cnvseq_drvdf.comments.append(cnv_drvdata_comment)
    
    
    
    cnv_da_process = Process(
        name = "VCF-DT",
        executes_protocol=study.protocols[14],
        parameter_values=[cnv_sw],
        inputs=[cnv_datafile],
        outputs=[cnvseq_drvdf]
    )
    
    
    assay_cnv_seq.process_sequence.append(extraction_process_cnv_seq)
    assay_cnv_seq.process_sequence.append(cnv_lib_process)
    assay_cnv_seq.process_sequence.append(cnv_seq_process)
    assay_cnv_seq.process_sequence.append(cnv_da_process)
    
    plink(sample_collection_gtx, extraction_process_cnv_seq)
    plink(extraction_process_cnv_seq, cnv_lib_process)
    plink(cnv_lib_process, cnv_seq_process)
    plink(cnv_seq_process, cnv_da_process)
#
assay_cnv_seq.characteristic_categories.append(char_ext_cnv_seq.category)
# print(assay_cnv_seq.other_material[0].characteristics[0].value.term)

[isatools.model.Characteristic(category=isatools.model.OntologyAnnotation(term='Stuff Type', term_source=None, term_accession='', comments=[]), value=isatools.model.OntologyAnnotation(term='gDNA', term_source=isatools.model.OntologySource(name='OBI', file='', version='', description='Ontology for Biomedical Investigations', comments=[]), term_accession='https://purl.org/OBOfoundry/obi:123414', comments=[]), unit=None, comments=[])]
[isatools.model.Characteristic(category=isatools.model.OntologyAnnotation(term='Stuff Type', term_source=None, term_accession='', comments=[]), value=isatools.model.OntologyAnnotation(term='gDNA', term_source=isatools.model.OntologySource(name='OBI', file='', version='', description='Ontology for Biomedical Investigations', comments=[]), term_accession='https://purl.org/OBOfoundry/obi:123414', comments=[]), unit=None, comments=[])]
[isatools.model.Characteristic(category=isatools.model.OntologyAnnotation(term='Stuff Type', term_source=None, term_accession=''

### Adding all ISA Assays declarations to the ISA Study object

In [18]:
# study.assays.append(assay)
#study.assays.append(assay_nmr_topo)
study.assays.append(assay_nmr_metpro)
study.assays.append(assay_rna_seq)
study.assays.append(assay_cnv_seq)

### Reporting a cross-technique integrative analysis by referencing a workflow (e.g. snakemake, galaxy) with an ISA protocol  and using ISA.Protocol.uri attribute to do so.

In [19]:
#Protocol #*
workflow_ref =Protocol(
    name="13C SIRM MS and NMR integrative analysis",
    description="a workflow for integrating data from NMR and MS acquisition into a consolidated result",
    uri="https://doi.org/10.1021/acs.analchem.1c01064",
    protocol_type=OntologyAnnotation(term="data transformation"),
    parameters=[
            ProtocolParameter(parameter_name=OntologyAnnotation(term="software"))
        ])
study.protocols.append(workflow_ref)

# print(investigation.ontology_source_references[4].comments[0])
# print(study.assays[3].comments[0])

### Serializing (writing) the ISA object representation to TAB with the ISA-API `dump` function

In [20]:
from isatools.isatab import dump

# note the use of the flag for explicit serialization on factor values on assay tables
dump(investigation, os.path.join(main_path,'TAB'), write_factor_values_in_assay_table=False)

isatools.model.Investigation(identifier='', filename='', title='', submission_date='', public_release_date='', ontology_source_references=[isatools.model.OntologySource(name='CHEBI', file='https://www.example.org/CHEBI', version='1.0', description='Chemical Entity of Biological Interest', comments=[]), isatools.model.OntologySource(name='EFO', file='', version='', description='Experimental Factor Ontology', comments=[]), isatools.model.OntologySource(name='OBI', file='', version='', description='Ontology for Biomedical Investigations', comments=[]), isatools.model.OntologySource(name='PATO', file='', version='', description='Phenotype and Trait Ontology', comments=[]), isatools.model.OntologySource(name='NCIBTaxon', file='', version='', description='NCBI Taxonomy', comments=[isatools.model.Comment(name='onto-test', value='onto-value')]), isatools.model.OntologySource(name='MSIO', file='', version='', description='Metabolomics Standards Initiative Ontology', comments=[]), isatools.model

### Loading ISA objects from TAB and writing back to TAB (roundtrip from TAB to TAB)

In [21]:
from isatools.isatab import load
with open(os.path.join(main_path,"TAB", "i_investigation.txt")) as isa_sirm_test:
    roundtrip = load(isa_sirm_test)

from isatools.isatab import dump

# note the use of the flag for explicit serialization on factor values on assay tables
dump(roundtrip, os.path.join(main_path,'TAB/BH23-ISATAB_FROM_TAB'), write_factor_values_in_assay_table=False)    
    

isatools.model.Investigation(identifier='', filename='', title='', submission_date='', public_release_date='', ontology_source_references=[isatools.model.OntologySource(name='CHEBI', file='https://www.example.org/CHEBI', version='1.0', description='Chemical Entity of Biological Interest', comments=[isatools.model.Comment(name='onto-test', value='')]), isatools.model.OntologySource(name='EFO', file='', version='', description='Experimental Factor Ontology', comments=[isatools.model.Comment(name='onto-test', value='')]), isatools.model.OntologySource(name='OBI', file='', version='', description='Ontology for Biomedical Investigations', comments=[isatools.model.Comment(name='onto-test', value='')]), isatools.model.OntologySource(name='PATO', file='', version='', description='Phenotype and Trait Ontology', comments=[isatools.model.Comment(name='onto-test', value='')]), isatools.model.OntologySource(name='NCIBTaxon', file='', version='', description='NCBI Taxonomy', comments=[isatools.model

### Serializing (writing) the ISA object representation to JSON with the ISA-API 

In [22]:
from isatools.isajson import dump
from isatools.isajson.dump import ISAJSONEncoder
from json import dumps, loads


inv_j = dumps(investigation, cls=ISAJSONEncoder)

with open(os.path.join(main_path, 'isa-bh2023-all.json'), 'w') as out_fp:
    out_fp.write(inv_j)
    
    
# with open(os.path.join(main_path, 'isa-bh2023-all.json')) as jsin_fp:
#     new_jsin = Investigation()
#     new_jsin.from_dict(json.loads(jsin_fp.read()))
    
# print(new_jsin.studies[0].assays[0])

### Validating the ISA object representation  with the ISA `validate` function

In [23]:
from isatools import isatab

my_json_report_isa_flux = isatab.validate(open(os.path.join(main_path,"TAB","i_investigation.txt")))

2024-03-11 15:23:42,958 [WARNING]: rules_10xx.py(check_protocol_usage:183) >> (W) Some protocols declared in the investigation file are not used neither in the study file s_BH2023-study.txt nor in any related assay file: ['MS metabolite identification', 'extracellular metabolite extraction', 'liquid chromatography mass spectrometry', '1D 13C NMR spectroscopy for isotopomer analysis', '13C SIRM MS and NMR integrative analysis']
2024-03-11 15:23:43,022 [WARNING]: rules_10xx.py(check_protocol_parameter_usage:263) >> (W) Some protocol parameters declared in the investigation file are not used in any assay file: ['mass analyzer', 'software', 'mass spectrometry instrument', 'ms software', 'chromatography column']
2024-03-11 15:23:43,039 [WARNING]: rules_40xx.py(check_protocol_fields:351) >> (W) Protocol(s) of type ['labeling'] defined in the ISA-configuration expected as a between 'Extract Name' and 'NMR Assay Name' but has not been found, in the file 'a_BH2023-metabolite-profiling-nmr-assay

cannot access local variable 'result' where it is not associated with a value
cannot access free variable 'spl' where it is not associated with a value in enclosing scope


In [24]:
my_json_report_isa_flux["errors"]

[]

NOTE: The error report indicates the need to add new configurations files matching the assay definitions.


### Loading from ISA-TAB from disk and converting to ISA-JSON

In [25]:
from isatools.isatab import load
with open(os.path.join(main_path,"TAB", "i_investigation.txt")) as isa_sirm_test:
    roundtrip = load(isa_sirm_test)

# print(roundtrip.studies[0].assays[0].samples[1].name)

# print(roundtrip.studies[0].assays[0].other_material[0].name)
# print(roundtrip.studies[0].assays[0].other_material[0].type)
# print(roundtrip.studies[0].assays[0].other_material[0].characteristics[0].value)
# #NOTE: this highlights an issue with the isatab load function,

# print(roundtrip.studies[0].assays[0].other_material[8].name)
# print(roundtrip.studies[0].assays[0].other_material[8].type)
# print(roundtrip.studies[0].assays[0].other_material[8].characteristics[0].value)

# print(roundtrip.studies[0].assays[1].other_material[1].name)
# print(roundtrip.studies[0].assays[1].other_material[1].type)
# print(roundtrip.studies[0].assays[1].other_material[1].characteristics[0].value) 

In [29]:
from isatools.convert import isatab2json
from isatools import isajson
import json

isa_json = isatab2json.convert(os.path.join(main_path, "TAB"), validate_first=True, use_new_parser=True)

print(isa_json["studies"][0]["assays"][0]["technologyType"]["annotationValue"])
print(isa_json["studies"][0]["assays"][0]["processSequence"][10]["name"])


output_path = os.path.join(main_path, 'JSON', 'isa-bh2023-t2j.json')
with open(output_path, 'w') as out_fp:
    json.dump(isa_json, out_fp)

with open(output_path) as out_fp:
    new_investigation_dict = json.loads(out_fp.read())
    new_investigation =  Investigation()
    new_investigation.from_dict(new_investigation_dict)
    print(new_investigation.studies[0].assays[1].process_sequence[0])


cannot access local variable 'result' where it is not associated with a value
cannot access free variable 'spl' where it is not associated with a value in enclosing scope
NMR spectroscopy
assay-name-nmr-metpro-CPMG-5
Process(name=)


###  Loading from ISA-JSON from disk and converting to ISA-TAB

In [27]:
from isatools.convert import json2isatab
with open(os.path.join(main_path,'JSON','isa-bh2023-t2j.json')) as in_fp:
    out_path = os.path.join(main_path,'JSON', 'BH23-ISATAB_FROM_JSON')
    print(out_path)
    json2isatab.convert(in_fp, out_path)
    

./output/ISA-BH2023-ALL/JSON/BH23-ISATAB_FROM_JSON
CONFIG at:  /Users/philippe/Documents/git/isa-api2/isa-api/isatools/isajson/../resources/config/json/default


###  Loading from PTMM ISA-JSON and converting to ISA-TAB (no assays in the input)

In [28]:
from isatools.convert import json2isatab
with open(os.path.join(main_path, 'isa-v2.json')) as in_fp:
    out_path = os.path.join(main_path,'JSON', 'BH23-ISATAB')
    json2isatab.convert(in_fp, out_path)

CONFIG at:  /Users/philippe/Documents/git/isa-api2/isa-api/isatools/isajson/../resources/config/json/default


## About this notebook

- authors: Philippe Rocca-Serra (philippe.rocca-serra@oerc.ox.ac.uk)
- license: CC-BY 4.0
- support: isatools@googlegroups.com
- issue tracker: https://github.com/ISA-tools/isa-api/issues